In [2]:
import laspy, numpy as np

IN_GT  = r"D:\lidarrrrr\anbu\OUTPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz"
OUT_GT = r"d:\lidarrrrr\anbu\test\GT_000001_MODEL.laz"

gt = laspy.read(IN_GT)
cls = np.array(gt.classification, dtype=np.int32)

REMAP = {
    0: 1,
    1: 1,
    2: 2,
    3: 3,
    4: 3,
    5: 6,   # ✅ IMPORTANT: Class 5 -> Building (6)
    6: 6,
    7: 1,
    9: 1,
    10: 1,
    12: 1,
    13: 1,
    14: 1,
    16: 1,
    17: 1,
    18: 6,
    19: 3,
    20: 3,
    21: 3,
    22: 3,
}

for k, v in REMAP.items():
    cls[cls == k] = v

gt.classification = cls.astype(np.uint8)
gt.write(OUT_GT)

print("After remap:", np.unique(cls))
print("Counts:", dict(zip(*np.unique(cls, return_counts=True))))

After remap: [1 2 3 6]
Counts: {np.int32(1): np.int64(581375), np.int32(2): np.int64(186001), np.int32(3): np.int64(665955), np.int32(6): np.int64(84601)}


In [3]:
import torch

MODEL_PATH = r"d:\lidarrrrr\anbu\dl_models\pointnetpp_best.pt"
ckpt = torch.load(MODEL_PATH, map_location="cpu")

print("Type:", type(ckpt))

if isinstance(ckpt, dict):
    print("Keys:", ckpt.keys())
    if "model_state" in ckpt:
        sd = ckpt["model_state"]
        print("\nFirst 50 layer names:")
        for i, k in enumerate(sd.keys()):
            print(k)
            if i >= 49:
                break
        # also show a few shapes (super useful)
        print("\nSome shapes:")
        shown = 0
        for k, v in sd.items():
            if hasattr(v, "shape"):
                print(k, tuple(v.shape))
                shown += 1
            if shown >= 15:
                break
else:
    # could be a full model
    print("This .pt seems to contain a full model object.")

Type: <class 'collections.OrderedDict'>
Keys: odict_keys(['mlp1.0.weight', 'mlp1.0.bias', 'mlp1.1.weight', 'mlp1.1.bias', 'mlp1.1.running_mean', 'mlp1.1.running_var', 'mlp1.1.num_batches_tracked', 'mlp1.3.weight', 'mlp1.3.bias', 'mlp1.4.weight', 'mlp1.4.bias', 'mlp1.4.running_mean', 'mlp1.4.running_var', 'mlp1.4.num_batches_tracked', 'mlp2.0.weight', 'mlp2.0.bias', 'mlp2.1.weight', 'mlp2.1.bias', 'mlp2.1.running_mean', 'mlp2.1.running_var', 'mlp2.1.num_batches_tracked', 'head.0.weight', 'head.0.bias', 'head.1.weight', 'head.1.bias', 'head.1.running_mean', 'head.1.running_var', 'head.1.num_batches_tracked', 'head.3.weight', 'head.3.bias'])


In [4]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# PATHS
# -----------------------------
MODEL_PATH = r"d:\lidarrrrr\anbu\dl_models\pointnetpp_best.pt"
GT_FILE    = r"d:\lidarrrrr\anbu\test\GT_000001_MODEL.laz"
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000001.laz"
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000001_PRED_PNPP.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# Make RAW from GT (same points)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    las = laspy.read(gt_path)
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# Load checkpoint (OrderedDict)
# -----------------------------
sd = torch.load(MODEL_PATH, map_location="cpu")  # OrderedDict
assert isinstance(sd, dict), "Expected state_dict (OrderedDict/dict)."
print("Loaded state_dict keys:", len(sd))

# Infer in/out channels from weights
# Conv1d weights are [out_ch, in_ch, 1]
w_mlp1_0 = sd["mlp1.0.weight"]
OUT1, IN_C, K = w_mlp1_0.shape
assert K == 1

w_head_3 = sd["head.3.weight"]
NUM_CLASSES = w_head_3.shape[0]
print("Model expects C =", IN_C, "| num_classes =", NUM_CLASSES)

# -----------------------------
# Define model matching keys
# -----------------------------
class PNPP_Like(nn.Module):
    def __init__(self, in_c, num_classes):
        super().__init__()
        # mlp1: Conv-BN-ReLU-Conv-BN-ReLU
        self.mlp1 = nn.Sequential(
            nn.Conv1d(in_c, OUT1, 1, bias=True),        # mlp1.0
            nn.BatchNorm1d(OUT1),                       # mlp1.1
            nn.ReLU(inplace=True),                      # mlp1.2 (not in sd)
            nn.Conv1d(OUT1, sd["mlp1.3.weight"].shape[0], 1, bias=True),  # mlp1.3
            nn.BatchNorm1d(sd["mlp1.3.weight"].shape[0]),                 # mlp1.4
            nn.ReLU(inplace=True),
        )

        # mlp2: Conv-BN-ReLU
        out_mlp2 = sd["mlp2.0.weight"].shape[0]
        in_mlp2  = sd["mlp2.0.weight"].shape[1]
        self.mlp2 = nn.Sequential(
            nn.Conv1d(in_mlp2, out_mlp2, 1, bias=True),  # mlp2.0
            nn.BatchNorm1d(out_mlp2),                    # mlp2.1
            nn.ReLU(inplace=True),
        )

        # head: Conv-BN-ReLU-Conv
        out_h0 = sd["head.0.weight"].shape[0]
        in_h0  = sd["head.0.weight"].shape[1]
        self.head = nn.Sequential(
            nn.Conv1d(in_h0, out_h0, 1, bias=True),      # head.0
            nn.BatchNorm1d(out_h0),                      # head.1
            nn.ReLU(inplace=True),
            nn.Conv1d(sd["head.3.weight"].shape[1], num_classes, 1, bias=True),  # head.3
        )

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = self.mlp1(x)
        x = self.mlp2(x)
        x = self.head(x)            # [B, num_classes, N]
        return x.permute(0, 2, 1)   # [B, N, num_classes]

model = PNPP_Like(IN_C, NUM_CLASSES).to(DEVICE)
model.load_state_dict(sd, strict=True)
model.eval()
print("✅ Model loaded OK")

# -----------------------------
# Feature builder (auto fit to C)
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    return np.hstack([X, np.zeros((X.shape[0], target_C - C), dtype=X.dtype)])

# -----------------------------
# Predict block-by-block
# -----------------------------
raw = laspy.read(RAW_FILE)
X_all = fit_C(build_features(raw), IN_C)

N = X_all.shape[0]
NPTS = 4096

idx = np.arange(N)
np.random.shuffle(idx)
blocks = []
for i in range(0, N, NPTS):
    b = idx[i:i+NPTS]
    if len(b) < NPTS:
        b = np.pad(b, (0, NPTS-len(b)), mode="wrap")
    blocks.append(b)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()
        xb[:,0] -= xb[:,0].mean()
        xb[:,1] -= xb[:,1].mean()
        xb[:,2] -= xb[:,2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)
        logits = model(xt)
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# REAL Accuracy (GT vs PRED)
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

labels = np.unique(y_true)  # only classes present in GT
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() else 0.0

print("\nREAL ACCURACY (PointNet++-pt)\n")
for c in labels:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Loaded state_dict keys: 30
Model expects C = 10 | num_classes = 5
✅ Model loaded OK
Predicting blocks: 371


100%|██████████| 371/371 [00:01<00:00, 225.88it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000001_PRED_PNPP.laz

REAL ACCURACY (PointNet++-pt)

Class 1 Accuracy (Recall): 35.76%
Class 2 Accuracy (Recall): 1.27%
Class 3 Accuracy (Recall): 0.00%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 33.36%


In [5]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# PATHS
# -----------------------------
MODEL_PATH = r"d:\lidarrrrr\anbu\dl_models\pointnetpp_best.pt"
GT_FILE    = r"d:\lidarrrrr\anbu\test\GT_000002_MODEL.laz"
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000002.laz"
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000002_PRED_PNPP.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# Make RAW from GT (same points)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    las = laspy.read(gt_path)
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# Load checkpoint (OrderedDict)
# -----------------------------
sd = torch.load(MODEL_PATH, map_location="cpu")  # OrderedDict
assert isinstance(sd, dict), "Expected state_dict (OrderedDict/dict)."
print("Loaded state_dict keys:", len(sd))

# Infer in/out channels from weights
# Conv1d weights are [out_ch, in_ch, 1]
w_mlp1_0 = sd["mlp1.0.weight"]
OUT1, IN_C, K = w_mlp1_0.shape
assert K == 1

w_head_3 = sd["head.3.weight"]
NUM_CLASSES = w_head_3.shape[0]
print("Model expects C =", IN_C, "| num_classes =", NUM_CLASSES)

# -----------------------------
# Define model matching keys
# -----------------------------
class PNPP_Like(nn.Module):
    def __init__(self, in_c, num_classes):
        super().__init__()
        # mlp1: Conv-BN-ReLU-Conv-BN-ReLU
        self.mlp1 = nn.Sequential(
            nn.Conv1d(in_c, OUT1, 1, bias=True),        # mlp1.0
            nn.BatchNorm1d(OUT1),                       # mlp1.1
            nn.ReLU(inplace=True),                      # mlp1.2 (not in sd)
            nn.Conv1d(OUT1, sd["mlp1.3.weight"].shape[0], 1, bias=True),  # mlp1.3
            nn.BatchNorm1d(sd["mlp1.3.weight"].shape[0]),                 # mlp1.4
            nn.ReLU(inplace=True),
        )

        # mlp2: Conv-BN-ReLU
        out_mlp2 = sd["mlp2.0.weight"].shape[0]
        in_mlp2  = sd["mlp2.0.weight"].shape[1]
        self.mlp2 = nn.Sequential(
            nn.Conv1d(in_mlp2, out_mlp2, 1, bias=True),  # mlp2.0
            nn.BatchNorm1d(out_mlp2),                    # mlp2.1
            nn.ReLU(inplace=True),
        )

        # head: Conv-BN-ReLU-Conv
        out_h0 = sd["head.0.weight"].shape[0]
        in_h0  = sd["head.0.weight"].shape[1]
        self.head = nn.Sequential(
            nn.Conv1d(in_h0, out_h0, 1, bias=True),      # head.0
            nn.BatchNorm1d(out_h0),                      # head.1
            nn.ReLU(inplace=True),
            nn.Conv1d(sd["head.3.weight"].shape[1], num_classes, 1, bias=True),  # head.3
        )

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = self.mlp1(x)
        x = self.mlp2(x)
        x = self.head(x)            # [B, num_classes, N]
        return x.permute(0, 2, 1)   # [B, N, num_classes]

model = PNPP_Like(IN_C, NUM_CLASSES).to(DEVICE)
model.load_state_dict(sd, strict=True)
model.eval()
print("✅ Model loaded OK")

# -----------------------------
# Feature builder (auto fit to C)
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    return np.hstack([X, np.zeros((X.shape[0], target_C - C), dtype=X.dtype)])

# -----------------------------
# Predict block-by-block
# -----------------------------
raw = laspy.read(RAW_FILE)
X_all = fit_C(build_features(raw), IN_C)

N = X_all.shape[0]
NPTS = 4096

idx = np.arange(N)
np.random.shuffle(idx)
blocks = []
for i in range(0, N, NPTS):
    b = idx[i:i+NPTS]
    if len(b) < NPTS:
        b = np.pad(b, (0, NPTS-len(b)), mode="wrap")
    blocks.append(b)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()
        xb[:,0] -= xb[:,0].mean()
        xb[:,1] -= xb[:,1].mean()
        xb[:,2] -= xb[:,2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)
        logits = model(xt)
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# REAL Accuracy (GT vs PRED)
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

labels = np.unique(y_true)  # only classes present in GT
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() else 0.0

print("\nREAL ACCURACY (PointNet++-pt)\n")
for c in labels:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Loaded state_dict keys: 30
Model expects C = 10 | num_classes = 5
✅ Model loaded OK
Predicting blocks: 853


100%|██████████| 853/853 [00:01<00:00, 447.88it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000002_PRED_PNPP.laz

REAL ACCURACY (PointNet++-pt)

Class 1 Accuracy (Recall): 63.93%
Class 2 Accuracy (Recall): 0.00%
Class 3 Accuracy (Recall): 0.00%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 18.27%


In [6]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# PATHS
# -----------------------------
MODEL_PATH = r"d:\lidarrrrr\anbu\dl_models\pointnetpp_best.pt"
GT_FILE    = r"d:\lidarrrrr\anbu\test\GT_000003_MODEL.laz"
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000003.laz"
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000003_PRED_PNPP.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# Make RAW from GT (same points)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    las = laspy.read(gt_path)
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# Load checkpoint (OrderedDict)
# -----------------------------
sd = torch.load(MODEL_PATH, map_location="cpu")  # OrderedDict
assert isinstance(sd, dict), "Expected state_dict (OrderedDict/dict)."
print("Loaded state_dict keys:", len(sd))

# Infer in/out channels from weights
# Conv1d weights are [out_ch, in_ch, 1]
w_mlp1_0 = sd["mlp1.0.weight"]
OUT1, IN_C, K = w_mlp1_0.shape
assert K == 1

w_head_3 = sd["head.3.weight"]
NUM_CLASSES = w_head_3.shape[0]
print("Model expects C =", IN_C, "| num_classes =", NUM_CLASSES)

# -----------------------------
# Define model matching keys
# -----------------------------
class PNPP_Like(nn.Module):
    def __init__(self, in_c, num_classes):
        super().__init__()
        # mlp1: Conv-BN-ReLU-Conv-BN-ReLU
        self.mlp1 = nn.Sequential(
            nn.Conv1d(in_c, OUT1, 1, bias=True),        # mlp1.0
            nn.BatchNorm1d(OUT1),                       # mlp1.1
            nn.ReLU(inplace=True),                      # mlp1.2 (not in sd)
            nn.Conv1d(OUT1, sd["mlp1.3.weight"].shape[0], 1, bias=True),  # mlp1.3
            nn.BatchNorm1d(sd["mlp1.3.weight"].shape[0]),                 # mlp1.4
            nn.ReLU(inplace=True),
        )

        # mlp2: Conv-BN-ReLU
        out_mlp2 = sd["mlp2.0.weight"].shape[0]
        in_mlp2  = sd["mlp2.0.weight"].shape[1]
        self.mlp2 = nn.Sequential(
            nn.Conv1d(in_mlp2, out_mlp2, 1, bias=True),  # mlp2.0
            nn.BatchNorm1d(out_mlp2),                    # mlp2.1
            nn.ReLU(inplace=True),
        )

        # head: Conv-BN-ReLU-Conv
        out_h0 = sd["head.0.weight"].shape[0]
        in_h0  = sd["head.0.weight"].shape[1]
        self.head = nn.Sequential(
            nn.Conv1d(in_h0, out_h0, 1, bias=True),      # head.0
            nn.BatchNorm1d(out_h0),                      # head.1
            nn.ReLU(inplace=True),
            nn.Conv1d(sd["head.3.weight"].shape[1], num_classes, 1, bias=True),  # head.3
        )

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = self.mlp1(x)
        x = self.mlp2(x)
        x = self.head(x)            # [B, num_classes, N]
        return x.permute(0, 2, 1)   # [B, N, num_classes]

model = PNPP_Like(IN_C, NUM_CLASSES).to(DEVICE)
model.load_state_dict(sd, strict=True)
model.eval()
print("✅ Model loaded OK")

# -----------------------------
# Feature builder (auto fit to C)
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    return np.hstack([X, np.zeros((X.shape[0], target_C - C), dtype=X.dtype)])

# -----------------------------
# Predict block-by-block
# -----------------------------
raw = laspy.read(RAW_FILE)
X_all = fit_C(build_features(raw), IN_C)

N = X_all.shape[0]
NPTS = 4096

idx = np.arange(N)
np.random.shuffle(idx)
blocks = []
for i in range(0, N, NPTS):
    b = idx[i:i+NPTS]
    if len(b) < NPTS:
        b = np.pad(b, (0, NPTS-len(b)), mode="wrap")
    blocks.append(b)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()
        xb[:,0] -= xb[:,0].mean()
        xb[:,1] -= xb[:,1].mean()
        xb[:,2] -= xb[:,2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)
        logits = model(xt)
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# REAL Accuracy (GT vs PRED)
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

labels = np.unique(y_true)  # only classes present in GT
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() else 0.0

print("\nREAL ACCURACY (PointNet++-pt)\n")
for c in labels:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Loaded state_dict keys: 30
Model expects C = 10 | num_classes = 5
✅ Model loaded OK
Predicting blocks: 552


100%|██████████| 552/552 [00:01<00:00, 460.69it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000003_PRED_PNPP.laz

REAL ACCURACY (PointNet++-pt)

Class 1 Accuracy (Recall): 43.13%
Class 2 Accuracy (Recall): 1.70%
Class 3 Accuracy (Recall): 0.00%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 14.40%


In [7]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# PATHS
# -----------------------------
MODEL_PATH = r"d:\lidarrrrr\anbu\dl_models\pointnetpp_best.pt"
GT_FILE    = r"d:\lidarrrrr\anbu\test\GT_000004_MODEL.laz"
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000004.laz"
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000004_PRED_PNPP.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# Make RAW from GT (same points)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    las = laspy.read(gt_path)
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# Load checkpoint (OrderedDict)
# -----------------------------
sd = torch.load(MODEL_PATH, map_location="cpu")  # OrderedDict
assert isinstance(sd, dict), "Expected state_dict (OrderedDict/dict)."
print("Loaded state_dict keys:", len(sd))

# Infer in/out channels from weights
# Conv1d weights are [out_ch, in_ch, 1]
w_mlp1_0 = sd["mlp1.0.weight"]
OUT1, IN_C, K = w_mlp1_0.shape
assert K == 1

w_head_3 = sd["head.3.weight"]
NUM_CLASSES = w_head_3.shape[0]
print("Model expects C =", IN_C, "| num_classes =", NUM_CLASSES)

# -----------------------------
# Define model matching keys
# -----------------------------
class PNPP_Like(nn.Module):
    def __init__(self, in_c, num_classes):
        super().__init__()
        # mlp1: Conv-BN-ReLU-Conv-BN-ReLU
        self.mlp1 = nn.Sequential(
            nn.Conv1d(in_c, OUT1, 1, bias=True),        # mlp1.0
            nn.BatchNorm1d(OUT1),                       # mlp1.1
            nn.ReLU(inplace=True),                      # mlp1.2 (not in sd)
            nn.Conv1d(OUT1, sd["mlp1.3.weight"].shape[0], 1, bias=True),  # mlp1.3
            nn.BatchNorm1d(sd["mlp1.3.weight"].shape[0]),                 # mlp1.4
            nn.ReLU(inplace=True),
        )

        # mlp2: Conv-BN-ReLU
        out_mlp2 = sd["mlp2.0.weight"].shape[0]
        in_mlp2  = sd["mlp2.0.weight"].shape[1]
        self.mlp2 = nn.Sequential(
            nn.Conv1d(in_mlp2, out_mlp2, 1, bias=True),  # mlp2.0
            nn.BatchNorm1d(out_mlp2),                    # mlp2.1
            nn.ReLU(inplace=True),
        )

        # head: Conv-BN-ReLU-Conv
        out_h0 = sd["head.0.weight"].shape[0]
        in_h0  = sd["head.0.weight"].shape[1]
        self.head = nn.Sequential(
            nn.Conv1d(in_h0, out_h0, 1, bias=True),      # head.0
            nn.BatchNorm1d(out_h0),                      # head.1
            nn.ReLU(inplace=True),
            nn.Conv1d(sd["head.3.weight"].shape[1], num_classes, 1, bias=True),  # head.3
        )

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = self.mlp1(x)
        x = self.mlp2(x)
        x = self.head(x)            # [B, num_classes, N]
        return x.permute(0, 2, 1)   # [B, N, num_classes]

model = PNPP_Like(IN_C, NUM_CLASSES).to(DEVICE)
model.load_state_dict(sd, strict=True)
model.eval()
print("✅ Model loaded OK")

# -----------------------------
# Feature builder (auto fit to C)
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    return np.hstack([X, np.zeros((X.shape[0], target_C - C), dtype=X.dtype)])

# -----------------------------
# Predict block-by-block
# -----------------------------
raw = laspy.read(RAW_FILE)
X_all = fit_C(build_features(raw), IN_C)

N = X_all.shape[0]
NPTS = 4096

idx = np.arange(N)
np.random.shuffle(idx)
blocks = []
for i in range(0, N, NPTS):
    b = idx[i:i+NPTS]
    if len(b) < NPTS:
        b = np.pad(b, (0, NPTS-len(b)), mode="wrap")
    blocks.append(b)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()
        xb[:,0] -= xb[:,0].mean()
        xb[:,1] -= xb[:,1].mean()
        xb[:,2] -= xb[:,2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)
        logits = model(xt)
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# REAL Accuracy (GT vs PRED)
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

labels = np.unique(y_true)  # only classes present in GT
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() else 0.0

print("\nREAL ACCURACY (PointNet++-pt)\n")
for c in labels:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Loaded state_dict keys: 30
Model expects C = 10 | num_classes = 5
✅ Model loaded OK
Predicting blocks: 45


100%|██████████| 45/45 [00:00<00:00, 159.71it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000004_PRED_PNPP.laz

REAL ACCURACY (PointNet++-pt)

Class 1 Accuracy (Recall): 66.37%
Class 2 Accuracy (Recall): 0.00%
Class 3 Accuracy (Recall): 0.00%
Class 6 Accuracy (Recall): 0.00%

Overall Accuracy: 47.96%


In [8]:
import os
import numpy as np
import laspy
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------
# PATHS
# -----------------------------
MODEL_PATH = r"d:\lidarrrrr\anbu\dl_models\pointnetpp_best.pt"
GT_FILE    = r"d:\lidarrrrr\anbu\test\GT_000005_MODEL.laz"
RAW_FILE   = r"D:\lidarrrrr\anbu\INPUT FILE\DX3013595 PASQUILIO\LAZ\DX3013595 PASQUILIO000005.laz"
PRED_FILE  = r"d:\lidarrrrr\anbu\test\RAW_000005_PRED_PNPP.laz"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# -----------------------------
# Make RAW from GT (same points)
# -----------------------------
def make_raw_from_gt(gt_path, raw_path):
    las = laspy.read(gt_path)
    las.classification[:] = 1
    las.write(raw_path)
    print("✅ RAW created:", raw_path)

if not os.path.exists(RAW_FILE):
    make_raw_from_gt(GT_FILE, RAW_FILE)

# -----------------------------
# Load checkpoint (OrderedDict)
# -----------------------------
sd = torch.load(MODEL_PATH, map_location="cpu")  # OrderedDict
assert isinstance(sd, dict), "Expected state_dict (OrderedDict/dict)."
print("Loaded state_dict keys:", len(sd))

# Infer in/out channels from weights
# Conv1d weights are [out_ch, in_ch, 1]
w_mlp1_0 = sd["mlp1.0.weight"]
OUT1, IN_C, K = w_mlp1_0.shape
assert K == 1

w_head_3 = sd["head.3.weight"]
NUM_CLASSES = w_head_3.shape[0]
print("Model expects C =", IN_C, "| num_classes =", NUM_CLASSES)

# -----------------------------
# Define model matching keys
# -----------------------------
class PNPP_Like(nn.Module):
    def __init__(self, in_c, num_classes):
        super().__init__()
        # mlp1: Conv-BN-ReLU-Conv-BN-ReLU
        self.mlp1 = nn.Sequential(
            nn.Conv1d(in_c, OUT1, 1, bias=True),        # mlp1.0
            nn.BatchNorm1d(OUT1),                       # mlp1.1
            nn.ReLU(inplace=True),                      # mlp1.2 (not in sd)
            nn.Conv1d(OUT1, sd["mlp1.3.weight"].shape[0], 1, bias=True),  # mlp1.3
            nn.BatchNorm1d(sd["mlp1.3.weight"].shape[0]),                 # mlp1.4
            nn.ReLU(inplace=True),
        )

        # mlp2: Conv-BN-ReLU
        out_mlp2 = sd["mlp2.0.weight"].shape[0]
        in_mlp2  = sd["mlp2.0.weight"].shape[1]
        self.mlp2 = nn.Sequential(
            nn.Conv1d(in_mlp2, out_mlp2, 1, bias=True),  # mlp2.0
            nn.BatchNorm1d(out_mlp2),                    # mlp2.1
            nn.ReLU(inplace=True),
        )

        # head: Conv-BN-ReLU-Conv
        out_h0 = sd["head.0.weight"].shape[0]
        in_h0  = sd["head.0.weight"].shape[1]
        self.head = nn.Sequential(
            nn.Conv1d(in_h0, out_h0, 1, bias=True),      # head.0
            nn.BatchNorm1d(out_h0),                      # head.1
            nn.ReLU(inplace=True),
            nn.Conv1d(sd["head.3.weight"].shape[1], num_classes, 1, bias=True),  # head.3
        )

    def forward(self, x):
        # x: [B, N, C] -> [B, C, N]
        x = x.permute(0, 2, 1)
        x = self.mlp1(x)
        x = self.mlp2(x)
        x = self.head(x)            # [B, num_classes, N]
        return x.permute(0, 2, 1)   # [B, N, num_classes]

model = PNPP_Like(IN_C, NUM_CLASSES).to(DEVICE)
model.load_state_dict(sd, strict=True)
model.eval()
print("✅ Model loaded OK")

# -----------------------------
# Feature builder (auto fit to C)
# -----------------------------
def build_features(las):
    dims = set(las.point_format.dimension_names)
    xyz = np.vstack([las.x, las.y, las.z]).T.astype(np.float32)

    intensity = np.array(las.intensity, dtype=np.float32) if "intensity" in dims else np.zeros(len(xyz), np.float32)
    rn = np.array(las.return_number, dtype=np.float32) if "return_number" in dims else np.ones(len(xyz), np.float32)
    nr = np.array(las.number_of_returns, dtype=np.float32) if "number_of_returns" in dims else np.ones(len(xyz), np.float32)

    if intensity.max() > intensity.min():
        intensity = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    ret_ratio = rn / (nr + 1e-6)

    X = np.column_stack([xyz, intensity, ret_ratio, nr]).astype(np.float32)  # C=6
    return X

def fit_C(X, target_C):
    C = X.shape[1]
    if C == target_C:
        return X
    if C > target_C:
        return X[:, :target_C]
    return np.hstack([X, np.zeros((X.shape[0], target_C - C), dtype=X.dtype)])

# -----------------------------
# Predict block-by-block
# -----------------------------
raw = laspy.read(RAW_FILE)
X_all = fit_C(build_features(raw), IN_C)

N = X_all.shape[0]
NPTS = 4096

idx = np.arange(N)
np.random.shuffle(idx)
blocks = []
for i in range(0, N, NPTS):
    b = idx[i:i+NPTS]
    if len(b) < NPTS:
        b = np.pad(b, (0, NPTS-len(b)), mode="wrap")
    blocks.append(b)

pred = np.zeros(N, dtype=np.int32)

print("Predicting blocks:", len(blocks))
with torch.no_grad():
    for b in tqdm(blocks):
        xb = X_all[b].copy()
        xb[:,0] -= xb[:,0].mean()
        xb[:,1] -= xb[:,1].mean()
        xb[:,2] -= xb[:,2].min()

        xt = torch.from_numpy(xb).float().unsqueeze(0).to(DEVICE)
        logits = model(xt)
        pb = logits.argmax(-1).squeeze(0).cpu().numpy().astype(np.int32)
        pred[b] = pb

# Save predicted LAZ
out = laspy.LasData(header=raw.header)
out.points = raw.points
out.classification = pred.astype(np.uint8)
out.write(PRED_FILE)
print("✅ Saved:", PRED_FILE)

# -----------------------------
# REAL Accuracy (GT vs PRED)
# -----------------------------
gt = laspy.read(GT_FILE)
y_true = np.array(gt.classification, dtype=np.int32)
y_pred = pred.astype(np.int32)

labels = np.unique(y_true)  # only classes present in GT
mask = np.isin(y_true, labels)

rep = classification_report(y_true[mask], y_pred[mask], labels=labels, digits=4, zero_division=0, output_dict=True)
cm  = confusion_matrix(y_true[mask], y_pred[mask], labels=labels)
overall = (np.trace(cm) / np.sum(cm)) * 100.0 if cm.sum() else 0.0

print("\nREAL ACCURACY (PointNet++-pt)\n")
for c in labels:
    print(f"Class {c} Accuracy (Recall): {rep[str(c)]['recall']*100:.2f}%")
print(f"\nOverall Accuracy: {overall:.2f}%")

Device: cuda
GPU: NVIDIA GeForce RTX 3050
Loaded state_dict keys: 30
Model expects C = 10 | num_classes = 5
✅ Model loaded OK
Predicting blocks: 430


100%|██████████| 430/430 [00:01<00:00, 408.75it/s]


✅ Saved: d:\lidarrrrr\anbu\test\RAW_000005_PRED_PNPP.laz

REAL ACCURACY (PointNet++-pt)

Class 1 Accuracy (Recall): 60.52%
Class 2 Accuracy (Recall): 0.73%
Class 3 Accuracy (Recall): 0.00%

Overall Accuracy: 22.55%
